In [1]:
# Set JAX_PLATFORMS=''
import os
os.environ["JAX_PLATFORMS"] = "cpu"
from inference.pipeline import DiffusionInferencePipeline

/home/mrwhite0racle/miniconda3/envs/flaxdiff/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-02 05:51:01.681267: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746165061.704537   94988 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746165061.711629   94988 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746165061.728424   94988 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target

In [2]:
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp

def normalizeImage(x): return jax.nn.standardize(x, mean=[127.5], std=[127.5])
def denormalizeImage(x): return (x + 1.0) * 127.5


def plotImages(imgs, fig_size=(8, 8), dpi=100):
    fig = plt.figure(figsize=fig_size, dpi=dpi)
    imglen = imgs.shape[0]
    for i in range(imglen):
        plt.subplot(fig_size[0], fig_size[1], i + 1)
        plt.imshow(jnp.astype(denormalizeImage(imgs[i, :, :, :]), jnp.uint8))
        plt.axis("off")
    plt.show()

In [3]:
pipeline = DiffusionInferencePipeline.from_wandb_registry(
    modelname="diffusion-oxford_flowers102-res128-sweep-d4es07fm",
    project='mlops-msml605-project',
    entity='umd-projects',
)


wandb: Downloading large artifact diffusion-oxford_flowers102-res128-sweep-d4es07fm:latest, 1552.72MB. 16 files... 
wandb:   16 of 16 files downloaded.  
Done. 0:0:2.0


Loaded model from wandb registry: diffusion-oxford_flowers102-res128-sweep-d4es07fm at path /home/mrwhite0racle/mlops-diffusion-project/artifacts/diffusion-oxford_flowers102-res128-sweep-d4es07fm:v21
Loaded checkpoint from local dir /home/mrwhite0racle/mlops-diffusion-project/artifacts/diffusion-oxford_flowers102-res128-sweep-d4es07fm:v21


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing FlaxCLIPTextModel: {('vision_model', 'encoder', 'layers', '21', 'mlp', 'fc1', 'kernel'), ('vision_model', 'encoder', 'layers', '23', 'self_attn', 'v_proj', 'bias'), ('vision_model', 'encoder', 'layers', '17', 'layer_norm2', 'scale'), ('vision_model', 'encoder', 'layers', '15', 'self_attn', 'q_proj', 'bias'), ('vision_model', 'encoder', 'layers', '16', 'mlp', 'fc2', 'kernel'), ('vision_model', 'encoder', 'layers', '3', 'layer_norm2', 'bias'), ('vision_model', 'encoder', 'layers', '11', 'self_attn', 'k_proj', 'kernel'), ('vision_model', 'encoder', 'layers', '3', 'mlp', 'fc2', 'kernel'), ('vision_model', 'encoder', 'layers', '13', 'mlp', 'fc2', 'bias'), ('vision_model', 'encoder', 'layers', '13', 'self_attn', 'q_proj', 'kernel'), ('vision_model', 'encoder', 'layers', '20', 'self_attn', 'v_proj', 'bias'), ('vision_model', 'encoder', 'layers', '1', 'self_attn', 'q_proj', 'bias'), ('vision_m

Model kwargs after mapping: {'dtype': <class 'jax.numpy.float32'>, 'precision': Precision.DEFAULT, 'activation': <PjitFunction of <function silu at 0x7f720f21f2e0>>, 'named_norms': False, 'norm_groups': 8, 'emb_features': 256, 'feature_depths': [64, 128, 256, 512], 'num_res_blocks': 3, 'output_channels': 3, 'attention_configs': [None, {'dtype': <class 'jax.numpy.float32'>, 'heads': 13, 'use_projection': False, 'flash_attention': False, 'use_self_and_cross': False, 'only_pure_attention': False}, {'dtype': <class 'jax.numpy.float32'>, 'heads': 13, 'use_projection': False, 'flash_attention': False, 'use_self_and_cross': False, 'only_pure_attention': False}, {'dtype': <class 'jax.numpy.float32'>, 'heads': 13, 'use_projection': False, 'flash_attention': False, 'use_self_and_cross': False, 'only_pure_attention': False}], 'num_middle_res_blocks': 1}


In [ ]:
prompts = [
    'water tulip',
    'a water lily',
    'a water lily',
    'a photo of a rose',
    'a photo of a rose',
    'a water lily',
    'a water lily',
    'a photo of a marigold',
]

samples = pipeline.generate_samples(
    num_samples=len(prompts),
    resolution=256,
    diffusion_steps=100,
    guidance_scale=3.0,
    start_step=1000,
    conditioning_data=prompts,
)

Created null embeddings for guidance with shape (1, 77, 768)
Using classifier-free guidance
Generating samples: steps=100, num_samples=8, guidance=3.0
Processing raw conditioning inputs to generate model conditioning inputs


 97%|█████████▋| 97/100 [09:18<00:17,  5.75s/it]

In [ ]:
plotImages(samples, dpi=500)

In [4]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
from flaxdiff.data.dataloaders import get_dataset_grain
data = get_dataset_grain(
    "oxford_flowers102", batch_size=BATCH_SIZE, image_scale=IMAGE_SIZE)
datalen = data['train_len']
batches = datalen // BATCH_SIZE

In [ ]:
import tqdm
import grain.python as pygrain
from flaxdiff.samplers.euler import EulerAncestralSampler

class EvaluationPipeline(DiffusionInferencePipeline):
    from torchmetrics.image.fid import FrechetInceptionDistance
    from torchmetrics.image.kid import KernelInceptionDistance
    from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.fid = self.FrechetInceptionDistance()
        self.kid = self.KernelInceptionDistance()
        self.lpip = self.LearnedPerceptualImagePatchSimilarity()
        
    def compute_metrics(self, original_images, generated_images):
        """
        Computes FID, KID, and LPIPS metrics between original and generated images.
        All images should be in the range [0, 255].
        """
        self.fid.update(original_images, real=True)  
        self.fid.update(generated_images, real=False)
        
        self.kid.update(original_images, real=True)
        self.kid.update(generated_images, real=False)
        
        # LPIPS expects images in the range [0, 1]
        original_images = original_images / 255.0
        generated_images = generated_images / 255.0
        self.lpip.update(original_images, generated=True)
        self.lpip.update(generated_images, generated=False)
        
        fid_score = self.fid.compute()
        kid_score = self.kid.compute()
        lpips_score = self.lpip.compute()
        
        return {
            'FID': fid_score,
            'KID': kid_score,
            'LPIPS': lpips_score
        }
    
    def evaluate(
        self, 
        dataloader: pygrain.DataLoader, 
        diffusion_steps=50, 
        sampler_class=EulerAncestralSampler,
    ):
        metrics_history = {}
        pbar = tqdm.tqdm(total=len(dataloader), desc="Evaluating")
        for batch in dataloader:
            original_images = batch['image']
            conditions = batch['text']
            model_conditioning_inputs = self.input_config.conditions[0].encoder.encode_from_tokens(conditions)
            
            generated = self.generate_samples(
                num_samples=BATCH_SIZE,
                resolution=IMAGE_SIZE,
                diffusion_steps=diffusion_steps,
                guidance_scale=3.0,
                start_step=1000,
                sampler_class=sampler_class,
                conditioning_data_tokens=model_conditioning_inputs
            )
            
            # Calculate metrics
            # Original images are in [0, 255]. Generated images are in [-1, 1]
            generated_255 = (generated * 127.5 + 127.5).astype(jnp.uint8)
            original_255 = original_images.astype(jnp.uint8)
            
            metrics = self.compute_metrics(original_255, generated_255)            
            for key, value in metrics.items():
                if key not in metrics_history:
                    metrics_history[key] = []
                metrics_history[key].append(value)
            pbar.update(1)
            pbar.set_postfix(metrics=metrics)
                
        # Average the metrics over all batches
        for key, value in metrics_history.items():
            metrics_history[key] = jnp.mean(jnp.array(value))
        return metrics_history